In [1]:
from calendar import c
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlr import MLR


Define an evaluation method

In [2]:
def calculateEvalMeasures(pred, targets, verbose = True):
    m = len(pred)
    mse = (((pred - targets)**2).sum() / (m))[0]
    mae = ((abs(pred - targets)).sum() / (m))[0]
    rmse = np.sqrt(mse)
    rss = (((targets-pred)**2).sum())[0]
    tss = (((targets-targets.mean())**2).sum())[0]
    r2 = 1 - (rss/tss)
    if verbose:
        print("R^2:", r2)
        print("Mean Absolute Error:", mae)
        print("MSE:", mse)
        print("RMSE:", rmse)
    return r2, mae, mse, rmse

Training performances using Autoscaling and Range-scaling

In [3]:
dataset = pd.read_csv("data/Folds5x2_pp.csv")
targetVarName = "PE"
mlr = MLR(dataset, targetVarName, "AUTO")
pred, params = mlr.learnParams(0.1)
# Training evaluation
targets = mlr.target
print("== Training results with Autoscaling ==")
r2, mae, mse, rmse = calculateEvalMeasures(pred, targets, True)

mlr = MLR(dataset, targetVarName, "MINMAX")
pred, params = mlr.learnParams(0.1)
# Training evaluation
targets = mlr.target
print("== Training results with Range-scaling (min-max) ==")
r2, mae, mse, rmse = calculateEvalMeasures(pred, targets, True)

== Training results with Autoscaling ==
R^2: 0.9254182191712128
Mean Absolute Error: 3.71315862837132
MSE: 21.72208350253979
RMSE: 4.66069560286228
== Training results with Range-scaling (min-max) ==
R^2: 0.8943045993614558
Mean Absolute Error: 4.401307406561619
MSE: 30.78398360821479
RMSE: 5.548331605826637


Cross-validation performances

In [4]:
def crossValidate(dataset, groupNumber, targetVarName, normMethod, verbose):
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    if groupNumber <= 1:
        raise Exception("Set at least 2 cross-validation groups")
    
    cvPartition = dataset.shape[0] // groupNumber
    sumR2, sumMae, sumMse, sumRmse = 0, 0, 0, 0
    for i in range(groupNumber):        
        initialRow = i * cvPartition
        finalRow = (i + 1) * cvPartition
        testSet = dataset[initialRow:finalRow].reset_index(drop=True)
        trainingSet = dataset.drop(dataset.index[initialRow:finalRow], axis = 0, inplace = False).reset_index(drop=True)
        testTargets = testSet[[targetVarName]]
        testSet = testSet.drop(targetVarName, axis=1)
        cvMlr = MLR(trainingSet, targetVarName, normMethod)
        trainingPred, params = cvMlr.learnParams(0.1)
        testPred = cvMlr.test(testSet)
        strOut = ""
        if normMethod == "AUTO":
            strOut = "Autoscaling"
        elif normMethod == "MINMAX":
            strOut = "Range scaling (min-max)"
        if verbose:
            print("== Cross-validation results with", strOut, " for group", i,"==")
        r2, mae, mse, rmse = calculateEvalMeasures(testPred, testTargets, verbose)
        sumR2 += r2
        sumMae += mae
        sumMse += mse
        sumRmse += rmse
    r2 = sumR2 / groupNumber
    mae = sumMae / groupNumber
    mse = sumMse / groupNumber
    rmse = sumRmse / groupNumber
    print("== Cross-validation results (average) with", strOut, "==")
    print("R^2:", r2)
    print("Mean Absolute Error:", mae)
    print("MSE:", mse)
    print("RMSE:", rmse)
    
    
crossValidate(dataset, 10, targetVarName, "AUTO", False)
crossValidate(dataset, 10, targetVarName, "MINMAX", False)

== Cross-validation results (average) with Autoscaling ==
R^2: 0.9237309236558033
Mean Absolute Error: 3.7484542038868
MSE: 22.16798763697417
RMSE: 4.706745423717078
== Cross-validation results (average) with Range scaling (min-max) ==
R^2: 0.8939908663138336
Mean Absolute Error: 4.384382593664254
MSE: 30.83427086559265
RMSE: 5.549760750419081
